In [1]:
import json
import openai
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

load_dotenv()

False

In [2]:


documents = []

with open('documents.json', 'r') as file:

    docs = json.load(file)
    
    

In [3]:
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
es_client.info()

ObjectApiResponse({'name': '3c78778d3d0a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'sMblFZ7TTVSam9nuOGxCfQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_settings = {"settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "faq_search"

es_client.indices.create(index = index_name, body = index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq_search'})

In [7]:
for doc in tqdm(documents):
    
    es_client.index(index = index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [11]:
def search(query, coursefilter = "data-engineering-zoomcamp"):

    search_settings =  {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": coursefilter
                    }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body = search_settings)

    result_docs = []
   
    #print(response)
    for hit in response['hits']['hits']:

        result_docs.append(hit['_source'])
    
    return result_docs

In [8]:
def build_prompt(query, related_docs):
    

    prompt = """
    You are an assistant for teaching online courses answer the question based on the context provided.
    Only the information available in the provided context should be used strictly adhere to this
    
    question:{query}
    
    context:{context}
    
    
    """.strip()
    
    
    context = ""
    for doc in related_docs:
        
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n\n"
        
    
    prompt = prompt.format(query = query, context = context).strip()
    
    return prompt

In [9]:
def chat(prompt, model = "phi3"):
   
    client = openai.OpenAI(
        base_url = "http://localhost:11434/v1/",
        
        api_key = "ollama"
    )
    
    response = client.chat.completions.create(
        model = model,
        messages = [{"role":"system", "content":"you are a faq assistant"},
                    {"role": "user", "content": prompt}]
    )
    
    print(response)
    
    return response.choices[0].message.content

In [10]:
def rag(query, model = "phi3"):

    results = search(query = query)
    prompt = build_prompt(query = query, related_docs = results)

    answer = chat(prompt, model = model)

    return answer

In [13]:
query = "I just found out the course has started can i still join it ?"
answer = rag( query = query)

InternalServerError: Error code: 500 - {'error': {'message': 'model requires more system memory (5.6 GiB) than is available (4.2 GiB)', 'type': 'api_error', 'param': None, 'code': None}}

In [ ]:
answer

'Yes, you can still join the course after the start date. You are eligible to submit homework, but be mindful of the deadlines for turning in final projects.'